In [1]:
import pytesseract
import cv2

In [2]:
image = cv2.imread('data/index_02.JPG')
base_image = image.copy()

In [3]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.imwrite('temp/index_gray.png', gray)

True

In [4]:
blur = cv2.GaussianBlur(gray, (7,7), 0)
cv2.imwrite('temp/index_blur.png', blur)

True

In [5]:
thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
cv2.imwrite('temp/index_thresh.png', thresh)

True

In [6]:
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,13))
cv2.imwrite('temp/index_kernel.png', kernel)

True

In [7]:
dilate = cv2.dilate(thresh, kernel, iterations=1)
cv2.imwrite('temp/index_dilate.png', dilate)

True

In [8]:
cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
cnts = sorted(cnts, key=lambda x: cv2.boundingRect(x)[0])

In [12]:
result = []
for c in cnts:
    x, y, w, h = cv2.boundingRect(c)
    if h > 200 and w> 20:
        roi = image[y:y+h, x:x+h]
        cv2.rectangle(image, (x, y), (x+w, y+h), (36,255,12), 2)
        ocr_result = pytesseract.image_to_string(roi)
        ocr_result = ocr_result.split('\n')
        for item in ocr_result:
            result.append(item)

cv2.imwrite("temp/index_bbox.png", image)

True

In [18]:
entities = []
for item in result:
    item = item.strip().replace("\n", "")
    item = item.split(" ")[0]
    if len(item) > 2 :
        if item[0] == "A" and "-" not in item:
            item = item.split(".")[0].replace(",","").replace(";","")
            entities.append(item)

In [ ]:
entities = list(set(entities))